In [ ]:
import sys

app = "/app"
if app not in sys.path:
    sys.path.append(app)
sys.path

In [ ]:
!pwd

In [ ]:
!ls /data/text/emboj_abstracts

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "text",
    data_dir="../data/text/emboj_abstracts/",
    data_files={'train': 'train/examples.txt', 'test': 'test/examples.txt'},
)
dataset

In [ ]:
MODEL_NAME = "roberta-base"

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"tokenizer vocab size: {tokenizer.vocab_size}")
print(f"max length: {tokenizer.max_model_input_sizes[MODEL_NAME]}")

In [ ]:
# tokenize the dataset
def tokenization(examples):
    return tokenizer(
    examples["text"],
        max_length=tokenizer.max_model_input_sizes[MODEL_NAME],
        truncation=True,
        return_special_tokens_mask=True,
    )
tokenized = dataset.map(tokenization, batched=True)
tokenized

In [ ]:
tokenized = tokenized.remove_columns(["text"])
tokenized

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer,
    mlm=True,
    mlm_probability=0.15
)

In [ ]:
from transformers import (
    # RobertaForMaskedLM,
    AutoConfig
)
from src.models.modeling_dendroberta import RobertaForMaskedLM

config = AutoConfig.from_pretrained(MODEL_NAME)
model = RobertaForMaskedLM(config=config)

In [ ]:
model_size = sum(t.numel() for t in model.parameters())
print(f"Model size: {model_size/1000**2:.1f}M parameters")

In [ ]:
from transformers import Trainer, TrainingArguments
from datetime import datetime

args = TrainingArguments(
    output_dir="../models",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="steps",
    eval_steps=500,
    prediction_loss_only=True,
    logging_steps=100,
    gradient_accumulation_steps=1,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    fp16=False,
    push_to_hub=False,
    logging_dir = f"../runs/lm-dendroberta-{datetime.now().isoformat().replace(':','-')}"
)

In [ ]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized["test"],
)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
trainer.train()

In [ ]:
# launch tensorboard
%tensorboard --logdir runs

In [ ]:
model.device